<a href="https://colab.research.google.com/github/ZetaRR/NY-apartments-EDA-Regression-project/blob/main/GettingData.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**Data Source Type:** Custom-made Zillow API

Link for API: https://rapidapi.com/apimaker/api/zillow-com1/



---



In [ ]:
from google.colab import drive, files
import json 
import pandas as pd
import requests
import time

pd.set_option('display.max_columns', 500)
pd.set_option('display.max_rows', 200)

In [ ]:
drive.mount('/drive', force_remount=False)

#keys =
city = 'New York'
state = 'NY'
search_query = city + ', ' + state
print(search_query)

Mounted at /drive
New York, NY


In [ ]:
api_key_df = pd.read_csv('/drive/MyDrive/df_to_csv/api_key.csv')
api_key = api_key_df.iloc[0,:].apply(str).values
api_key = ' '.join([str(i) for i in api_key]) 

In [ ]:
#FIRST PAGE
url = "https://zillow-com1.p.rapidapi.com/propertyExtendedSearch"

querystring = {"location":search_query,"home_type":"Apartments",
               "page":"1"}

headers = {
	"X-RapidAPI-Host": "zillow-com1.p.rapidapi.com",
	"X-RapidAPI-Key": api_key,
  "minPrice":'6000000',
  "sqftMin":'150'
}

response = requests.request("GET", url, headers=headers, params=querystring)
response = response.json()
#response

In [ ]:
pd.set_option('display.max_columns', 500)
df_first_page = pd.json_normalize(data=response['props'])
print('Columns -->', df_first_page.columns)
print('Nuw of Columns -->', len(df_first_page.columns))
print('Num of rows -->', len(df_first_page) )
df_first_page.head(2)

Columns --> Index(['dateSold', 'propertyType', 'lotAreaValue', 'address', 'daysOnZillow',
       'price', 'listingDateTime', 'longitude', 'latitude',
       'contingentListingType', 'listingStatus', 'zpid', 'imgSrc',
       'livingArea', 'bathrooms', 'lotAreaUnit', 'country', 'currency',
       'bedrooms', 'hasImage', 'listingSubType.is_FSBA',
       'listingSubType.is_openHouse'],
      dtype='object')
Nuw of Columns --> 22
Num of rows --> 40


,dateSold,propertyType,lotAreaValue,address,daysOnZillow,price,listingDateTime,longitude,latitude,contingentListingType,listingStatus,zpid,imgSrc,livingArea,bathrooms,lotAreaUnit,country,currency,bedrooms,hasImage,listingSubType.is_FSBA,listingSubType.is_openHouse
0,None,APARTMENT,NaN,"474 48th Ave APT 29B, Long Island City, NY 11109",-1,710000,None,-73.95634,40.744408,None,FOR_SALE,79739498,https://photos.zillowstatic.com/fp/93f19aea44f...,1046.0,2,None,USA,USD,2,True,True,NaN
1,None,APARTMENT,NaN,"4-74 48th Ave PENTHOUSE 1C, Long Island City, ...",-1,555555,None,-73.95634,40.744408,None,FOR_SALE,219836786,https://photos.zillowstatic.com/fp/df7f9142c3c...,921.0,1,None,USA,USD,1,True,True,NaN


In [ ]:
#EXTENDED SEARCH
zpids = df_first_page['zpid'].tolist()

detail_list = []
for zpid in zpids:
  url = "https://zillow-com1.p.rapidapi.com/property"

  querystring = {"zpid":zpid}

  headers = {
	"X-RapidAPI-Host": "zillow-com1.p.rapidapi.com",
	"X-RapidAPI-Key": api_key
  }

  response_zpids = requests.request("GET", url, headers=headers, params=querystring)
  response_zpids_json = response_zpids.json()
  time.sleep(6.1)

  detail_list.append(response_zpids_json)

In [ ]:
details_cols = [
      'propertyTaxRate',
      'timeOnZillow',
      'isListedByOwner',
      'zipcode',
      'annualHomeownersInsurance',
      'rentZestimate',
      'yearBuilt',
      'pageViewCount',
      'listingProvider.postingGroupName',
      'resoFacts.cityRegion',
      'resoFacts.hasCooling',
      'resoFacts.hasView']

df_prop_details = pd.json_normalize(data=detail_list)
df_prop_details = df_prop_details[details_cols]

df_first_page = pd.concat([df_first_page, df_prop_details], axis=1)
df_first_page.head(5)

,dateSold,propertyType,lotAreaValue,address,daysOnZillow,price,listingDateTime,longitude,latitude,contingentListingType,listingStatus,zpid,imgSrc,livingArea,bathrooms,lotAreaUnit,country,currency,bedrooms,hasImage,listingSubType.is_FSBA,listingSubType.is_openHouse,propertyTaxRate,timeOnZillow,isListedByOwner,zipcode,annualHomeownersInsurance,rentZestimate,yearBuilt,pageViewCount,listingProvider.postingGroupName,resoFacts.cityRegion,resoFacts.hasCooling,resoFacts.hasView
0,None,APARTMENT,NaN,"474 48th Ave APT 29B, Long Island City, NY 11109",-1,710000,None,-73.956340,40.744408,None,FOR_SALE,79739498,https://photos.zillowstatic.com/fp/93f19aea44f...,1046.0,2,None,USA,USD,2,True,True,NaN,0.84,177 days,False,11109,2982,3319.0,1996.0,7675,Modern Spaces,Long Island City,False,True
1,None,APARTMENT,NaN,"4-74 48th Ave PENTHOUSE 1C, Long Island City, ...",-1,555555,None,-73.956340,40.744408,None,FOR_SALE,219836786,https://photos.zillowstatic.com/fp/df7f9142c3c...,921.0,1,None,USA,USD,1,True,True,NaN,0.84,640 days,False,11109,2333,3776.0,1996.0,2317,Modern Spaces,Long Island City,True,True
2,None,APARTMENT,NaN,"4-74 48th Ave #21AB, Long Island City, NY 11109",-1,1250000,None,-73.956800,40.744200,None,FOR_SALE,2078556904,https://photos.zillowstatic.com/fp/6a9ac7ac0b6...,2200.0,2,None,USA,USD,3,True,True,NaN,0.84,18 days,False,11109,5250,5351.0,1996.0,3558,Compass,Long Island City,False,False
3,None,APARTMENT,NaN,"1210 Evergreen Ave APT 4, Bronx, NY 10472",-1,299000,None,-73.880875,40.828934,None,FOR_SALE,2069656047,https://photos.zillowstatic.com/fp/9626a7e4250...,1100.0,1,None,USA,USD,2,True,True,NaN,0.95,56 days,False,10472,1256,1975.0,1913.0,2746,"The Leveque Group, LLC",Bronx,False,False
4,None,APARTMENT,NaN,"334 E 65th St APT 42, New York, NY 10065",-1,399000,None,-73.960500,40.763600,None,FOR_SALE,2067837641,https://photos.zillowstatic.com/fp/7b54dfcb572...,3809.0,1,None,USA,USD,1,True,True,NaN,0.88,173 days,False,10065,1676,3930.0,1881.0,4101,Engel & Völkers New York Real Estate,New York,True,False


In [ ]:
#SECOND PAGE
url = "https://zillow-com1.p.rapidapi.com/propertyExtendedSearch"

querystring = {"location":search_query,"home_type":"Apartments",
               "page":"2"}

headers = {
	"X-RapidAPI-Host": "zillow-com1.p.rapidapi.com",
	"X-RapidAPI-Key": api_key,
  "minPrice":'6000000',
  "sqftMin":'150'
}

response = requests.request("GET", url, headers=headers, params=querystring)
response = response.json()
#response

In [ ]:
df_second_page = pd.json_normalize(data=response['props'])
print('Columns -->', df_second_page.columns)
print('Nuw of Columns -->', len(df_second_page.columns))
print('Num of rows -->', len(df_second_page) )
df_second_page.head(5)

Columns --> Index(['dateSold', 'propertyType', 'lotAreaValue', 'address', 'daysOnZillow',
       'price', 'listingDateTime', 'longitude', 'latitude',
       'contingentListingType', 'listingStatus', 'zpid', 'imgSrc',
       'livingArea', 'bathrooms', 'lotAreaUnit', 'country', 'currency',
       'bedrooms', 'hasImage', 'listingSubType.is_FSBA',
       'listingSubType.is_foreclosure'],
      dtype='object')
Nuw of Columns --> 22
Num of rows --> 40


,dateSold,propertyType,lotAreaValue,address,daysOnZillow,price,listingDateTime,longitude,latitude,contingentListingType,listingStatus,zpid,imgSrc,livingArea,bathrooms,lotAreaUnit,country,currency,bedrooms,hasImage,listingSubType.is_FSBA,listingSubType.is_foreclosure
0,None,APARTMENT,NaN,"19-19 24th Ave APT L117, Astoria, NY 11105",-1,699000,None,-73.922230,40.777310,None,FOR_SALE,81860691,https://photos.zillowstatic.com/fp/63b074c2c58...,1075.0,1,None,USA,USD,1,True,True,NaN
1,None,APARTMENT,NaN,"520 W 23rd St APT 5E, New York, NY 10011",-1,730000,None,-74.004814,40.747807,None,FOR_SALE,72514862,https://photos.zillowstatic.com/fp/88880419271...,608.0,1,None,USA,USD,1,True,True,NaN
2,None,APARTMENT,NaN,"8794 17th Ave #3D, Brooklyn, NY 11214",-1,675000,None,-74.009254,40.604782,None,FOR_SALE,2065747394,https://photos.zillowstatic.com/fp/523123c76fd...,925.0,2,None,USA,USD,2,True,True,NaN
3,None,APARTMENT,NaN,"63 E 9th St APT 12H, New York, NY 10003",-1,1650000,None,-73.992070,40.731583,None,FOR_SALE,244744707,https://photos.zillowstatic.com/fp/417b15e165c...,NaN,2,None,USA,USD,3,True,True,NaN
4,None,APARTMENT,NaN,"160 S 4th St #1, Brooklyn, NY 11211",-1,3790000,None,-73.961754,40.711273,None,FOR_SALE,2065498544,https://photos.zillowstatic.com/fp/9389931c507...,2795.0,4,None,USA,USD,5,True,True,NaN


In [ ]:
#EXTENDED SEARCH
zpids = df_second_page['zpid'].tolist()

detail_list = []
for zpid in zpids:
  url = "https://zillow-com1.p.rapidapi.com/property"

  querystring = {"zpid":zpid}

  headers = {
	"X-RapidAPI-Host": "zillow-com1.p.rapidapi.com",
	"X-RapidAPI-Key": api_key
  }

  response_zpids = requests.request("GET", url, headers=headers, params=querystring)
  response_zpids_json = response_zpids.json()
  time.sleep(6.1)

  detail_list.append(response_zpids_json)

In [ ]:
df_prop_details = pd.json_normalize(data=detail_list)
df_prop_details = df_prop_details[details_cols]

df_second_page = pd.concat([df_second_page, df_prop_details], axis=1)
df_second_page.head(5)

,dateSold,propertyType,lotAreaValue,address,daysOnZillow,price,listingDateTime,longitude,latitude,contingentListingType,listingStatus,zpid,imgSrc,livingArea,bathrooms,lotAreaUnit,country,currency,bedrooms,hasImage,listingSubType.is_FSBA,listingSubType.is_foreclosure,propertyTaxRate,timeOnZillow,isListedByOwner,zipcode,annualHomeownersInsurance,rentZestimate,yearBuilt,pageViewCount,listingProvider.postingGroupName,resoFacts.cityRegion,resoFacts.hasCooling,resoFacts.hasView
0,None,APARTMENT,NaN,"19-19 24th Ave APT L117, Astoria, NY 11105",-1,699000,None,-73.922230,40.777310,None,FOR_SALE,81860691,https://photos.zillowstatic.com/fp/63b074c2c58...,1075.0,1,None,USA,USD,1,True,True,NaN,0.84,5 days,False,11105,2936.0,2939.0,2007.0,222.0,Triplemint,Astoria,False,False
1,None,APARTMENT,NaN,"520 W 23rd St APT 5E, New York, NY 10011",-1,730000,None,-74.004814,40.747807,None,FOR_SALE,72514862,https://photos.zillowstatic.com/fp/88880419271...,608.0,1,None,USA,USD,1,True,True,NaN,0.88,87 days,False,10011,3066.0,5450.0,2003.0,661.0,Compass,New York,True,False
2,None,APARTMENT,NaN,"8794 17th Ave #3D, Brooklyn, NY 11214",-1,675000,None,-74.009254,40.604782,None,FOR_SALE,2065747394,https://photos.zillowstatic.com/fp/523123c76fd...,925.0,2,None,USA,USD,2,True,True,NaN,0.65,40 days,False,11214,2835.0,NaN,2017.0,1480.0,Ilite Realty Inc,Brooklyn,True,False
3,None,APARTMENT,NaN,"63 E 9th St APT 12H, New York, NY 10003",-1,1650000,None,-73.992070,40.731583,None,FOR_SALE,244744707,https://photos.zillowstatic.com/fp/417b15e165c...,NaN,2,None,USA,USD,3,True,True,NaN,0.88,60 days,False,10003,6930.0,4697.0,1955.0,2023.0,Brown Harris Stevens,New York,True,False
4,None,APARTMENT,NaN,"160 S 4th St #1, Brooklyn, NY 11211",-1,3790000,None,-73.961754,40.711273,None,FOR_SALE,2065498544,https://photos.zillowstatic.com/fp/9389931c507...,2795.0,4,None,USA,USD,5,True,True,NaN,0.65,20 days,False,11211,15918.0,NaN,1899.0,1381.0,Sotheby's International Realty,Brooklyn,True,False


In [ ]:
url = "https://zillow-com1.p.rapidapi.com/propertyExtendedSearch"

querystring = {"location":search_query,"home_type":"Apartments",
               "page":"3"}

headers = {
	"X-RapidAPI-Host": "zillow-com1.p.rapidapi.com",
	"X-RapidAPI-Key": api_key,
  "minPrice":'6000000',
  "sqftMin":'150'
}

response = requests.request("GET", url, headers=headers, params=querystring)
response = response.json()

In [ ]:
pd.set_option('display.max_columns', 500)
df_third_page = pd.json_normalize(data=response['props'])
print('Columns -->', df_third_page.columns)
print('Nuw of Columns -->', len(df_third_page.columns))
print('Num of rows -->', len(df_third_page) )
df_third_page.head(5)

KeyError: ignored

In [ ]:
zpids = df_third_page['zpid'].tolist()

detail_list = []
for zpid in zpids:
  url = "https://zillow-com1.p.rapidapi.com/property"

  querystring = {"zpid":zpid}

  headers = {
	"X-RapidAPI-Host": "zillow-com1.p.rapidapi.com",
	"X-RapidAPI-Key": api_key
  }

  response_zpids = requests.request("GET", url, headers=headers, params=querystring)
  response_zpids_json = response_zpids.json()
  time.sleep(6.1)

  detail_list.append(response_zpids_json)

In [ ]:
df_prop_details = pd.json_normalize(data=detail_list)
df_prop_details = df_prop_details[details_cols]

df_third_page = pd.concat([df_third_page, df_prop_details], axis=1)
df_third_page.head(5)

In [ ]:
url = "https://zillow-com1.p.rapidapi.com/propertyExtendedSearch"

querystring = {"location":search_query,"home_type":"Apartments",
               "page":"4"}

headers = {
	"X-RapidAPI-Host": "zillow-com1.p.rapidapi.com",
	"X-RapidAPI-Key": api_key,
  "minPrice":'6000000',
  "sqftMin":'150'
}

response = requests.request("GET", url, headers=headers, params=querystring)
response = response.json()

In [ ]:
pd.set_option('display.max_columns', 500)
df_fourth_page = pd.json_normalize(data=response['props'])
print('Columns -->', df_fourth_page.columns)
print('Nuw of Columns -->', len(df_fourth_page.columns))
print('Num of rows -->', len(df_fourth_page) )
df_fourth_page.head(5)

In [ ]:
zpids = df_fourth_page['zpid'].tolist()

detail_list = []
for zpid in zpids:
  url = "https://zillow-com1.p.rapidapi.com/property"

  querystring = {"zpid":zpid}

  headers = {
	"X-RapidAPI-Host": "zillow-com1.p.rapidapi.com",
	"X-RapidAPI-Key": api_key
  }

  response_zpids = requests.request("GET", url, headers=headers, params=querystring)
  response_zpids_json = response_zpids.json()
  time.sleep(6.1)

  detail_list.append(response_zpids_json)

In [ ]:
df_prop_details = pd.json_normalize(data=detail_list)
df_prop_details = df_prop_details[details_cols]

df_fourth_page = pd.concat([df_fourth_page, df_prop_details], axis=1)
df_third_page.head(5)

In [ ]:
#concating all dataframes
df = pd.concat([df_first_page, df_second_page, df_third_page, df_fourth_page],axis=0)

In [ ]:
df.to_csv('/drive/MyDrive/df_to_csv/zillow_df.csv', encoding='utf-8', index=False) # saving df on my own Google Drive